In [ ]:
!pip3 install virtualenv

In [ ]:
!virtualenv perspectiveAPI

In [4]:
!source perspectiveAPI/bin/activate

In [ ]:
!perspectiveAPI/bin/pip install google-api-python-client

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!/root/google-cloud-sdk/bin/gcloud init

In [8]:
!gcloud services enable commentanalyzer.googleapis.com

In [ ]:
from google.colab import files

uploaded = files.upload()

In [12]:
import pandas as pd
from googleapiclient import discovery
from IPython.display import display

# Define your API_KEY
API_KEY = "API_KEY"

# Initialize the client
client = discovery.build(
    "commentanalyzer",
    "v1alpha1",
    developerKey=API_KEY,
    discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
    static_discovery=False,
)

# Function to analyze text and return results
def analyze_text(text):
    try:
        analyze_request = {
            'comment': {'text': text},
            'requestedAttributes': {
                'TOXICITY': {},
                'IDENTITY_ATTACK': {},
                'INSULT': {},
                'PROFANITY': {},
                'THREAT': {}
            }
        }
        response = client.comments().analyze(body=analyze_request).execute()
        return response
    except Exception as e:
        error_details = e.error_details[0]
        if 'LANGUAGE_NOT_SUPPORTED_BY_ATTRIBUTE' in error_details:
            log_message = f"Skipping analysis for text due to unsupported language: {text}"
        elif 'LANGUAGE_NOT_SUPPORTED' in error_details:
            log_message = f"Skipping analysis for text due to undefined language: {text}"
        else:
            log_message = f"Skipping analysis due to error: {error_details}"

        # Log the error message
        with open("analysis_errors.log", "a") as log_file:
            log_file.write(log_message + "\n")

        # Return default values of 0 for each category
        return {
            'attributeScores': {
                'TOXICITY': {'summaryScore': {'value': 0}},
                'IDENTITY_ATTACK': {'summaryScore': {'value': 0}},
                'INSULT': {'summaryScore': {'value': 0}},
                'PROFANITY': {'summaryScore': {'value': 0}},
                'THREAT': {'summaryScore': {'value': 0}}
            }
        }

# Load CSV file into a DataFrame with the correct delimiter
df = pd.read_csv('VOX_Videos_2023.csv', delimiter=';')

# Replace NaN values in 'videoDescription' with an empty string
df['videoDescription'] = df['videoDescription'].fillna('')

# Define an empty list to store results
results = []

# Analyze each text and store the results
for index, row in df.iterrows():
    text = row['videoDescription']
    #print(f"Analyzing text at row {index+1}...")
    response = analyze_text(text)
    result = {}
    for attribute, scores in response['attributeScores'].items():
        score_value = scores['summaryScore']['value']
        result[attribute] = score_value
        # Extract spanScores values if present
        if 'spanScores' in scores:
            for score in scores['spanScores']:
                score_type = score['score']['type']
                score_value = score['score']['value']
                result[f'{attribute}_{score_type}'] = score_value
    results.append(result)

# Create a DataFrame to store the results
results_df = pd.DataFrame(results)

# Combine the 'videoDescription' column with the DataFrame
results_df['videoDescription'] = df['videoDescription']

# Reorder columns
columns_order = ['videoDescription'] + sorted(results_df.columns[:-1])  # Exclude the 'videoDescription' column
results_df = results_df[columns_order]

# Save the DataFrame to a CSV file
results_df.to_csv('VOX_perspective_results.csv', index=False)

# Display the results DataFrame
display(results_df)


,videoDescription,IDENTITY_ATTACK,IDENTITY_ATTACK_PROBABILITY,INSULT,INSULT_PROBABILITY,PROFANITY,PROFANITY_PROBABILITY,THREAT,THREAT_PROBABILITY,TOXICITY,TOXICITY_PROBABILITY
0,Desde VOX deseamos que todos los españoles enc...,0.002081,0.002081,0.007703,0.007703,0.009520,0.009520,0.005651,0.005651,0.007351,0.007351
1,"El secretario general de VOX, Ignacio Garriga,...",0.001239,0.001239,0.010249,0.010249,0.009409,0.009409,0.005437,0.005437,0.009550,0.009550
2,"El presidente nacional de VOX, Santiago Abasca...",0.000110,0.000110,0.005067,0.005067,0.007706,0.007706,0.005207,0.005207,0.000668,0.000668
3,Discurso completo del presidente nacional de V...,0.009101,0.009101,0.182317,0.182317,0.038405,0.038405,0.006104,0.006104,0.191243,0.191243
4,En Roma junto a nuestros amigos de Fratelli d'...,0.005513,0.005513,0.010401,0.010401,0.011458,0.011458,0.005945,0.005945,0.014011,0.014011
...,...,...,...,...,...,...,...,...,...,...,...
597,https://www.voxespana.es/actualidad/vox-llevar...,0.015858,0.015858,0.034783,0.034783,0.041161,0.041161,0.010939,0.010939,0.093081,0.093081
598,Declaraciones de Santiago Abascal ante el Trib...,0.003589,0.003589,0.017421,0.017421,0.013234,0.013234,0.005473,0.005473,0.018723,0.018723
599,"Santiago Abascal, presidente de VOX, y diputad...",0.004125,0.004125,0.018001,0.018001,0.012175,0.012175,0.005761,0.005761,0.024378,0.024378
600,https://www.voxespana.es/noticias/vox-presenta...,0.016762,0.016762,0.032579,0.032579,0.026864,0.026864,0.010395,0.010395,0.087956,0.087956


In [ ]:
from google.colab import files
files.download('example_perspective_video_description.csv')